<a href="https://colab.research.google.com/github/ronincodex/Student-Engagement-and-Performance-SOM/blob/main/copy_of_student_engagement_and_performance_som.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import seaborn as sn

from numpy.ma.core import ceil
from scipy.spatial import distance #distance calculation
from sklearn.preprocessing import MinMaxScaler #normalisation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score #scoring
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib import animation, colors
from sklearn.decomposition import PCA
import pandas as pd

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
# Engagement Data Set
data_engagement = "Student_Engagement.csv"
data_performance = "Student_performance.csv"

# Performance

## Read Data

In [ ]:
data_performance = pd.read_csv('/content/sample_data/Student_Performance.csv')
data_performance = data_performance.drop(["Student ID"],axis=1)
data_performance.head()

## Check for missing value

In [ ]:
msno.matrix(data_performance,figsize=(10,3))


## Correlation between Features

In [ ]:
continiousData = data_performance.iloc[:,:-1]
continiousData.head()

fig, axes = plt.subplots(nrows=1,ncols=1)
fig.set_size_inches(20, 15)
sn.boxplot(data=continiousData,orient="v",ax=axes)

In [ ]:
# Correlation analysis
corrMatt = data_performance.corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sn.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)

## Normalization

In [ ]:
# normalization
def minmax_scaler(data):
  scaler = MinMaxScaler()
  scaled = scaler.fit_transform(data)
  return scaled

normalized_performance=minmax_scaler(data_performance.iloc[:,:-1])
fig, axes = plt.subplots(nrows=1,ncols=1)
fig.set_size_inches(25, 15)
sn.boxplot(data=normalized_performance,orient="v",ax=axes)

## Clustering via SOM

In [ ]:
## SOM

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
print(tf.__version__)

class SOM(object): 
  
  #Initialize properties
  def __init__(self, x, y, input_dim, learning_rate, radius, num_iter=111):
    self._x = x
    self._y = y
    self._learning_rate = float(learning_rate)
    self._radius = float(radius)
    self._num_iter = num_iter
    self._graph = tf.Graph()

    #Initialize graph
    with self._graph.as_default():

        #Initializing variables and placeholders
        self._weights = tf.Variable(tf.random_normal([x*y, input_dim]))
        self._locations = self._generate_index_matrix(x, y)
        self._input = tf.placeholder("float", [input_dim])
        self._iter_input = tf.placeholder("float")

        #Calculating BMU
        input_matix = tf.stack([self._input for i in range(x*y)])
        distances = tf.sqrt(tf.reduce_sum(tf.pow(tf.subtract(self._weights, input_matix), 2), 1))
        bmu = tf.argmin(distances, 0)

        #Get BMU location
        mask = tf.pad(tf.reshape(bmu, [1]), np.array([[0, 1]]))
        size = tf.cast(tf.constant(np.array([1, 2])), dtype=tf.int64)
        bmu_location = tf.reshape(tf.slice(self._locations, mask, size), [2])

        #Calculate learning rate and radius
        decay_function = tf.subtract(1.0, tf.div(self._iter_input, self._num_iter))
        _current_learning_rate = tf.multiply(self._learning_rate, decay_function)
        _current_radius = tf.multiply(self._radius, decay_function)

        #Adapt learning rate to each neuron based on position
        bmu_matrix = tf.stack([bmu_location for i in range(x*y)])
        bmu_distance = tf.reduce_sum(tf.pow(tf.subtract(self._locations, bmu_matrix), 2), 1)
        neighbourhood_func = tf.exp(tf.negative(tf.div(tf.cast(bmu_distance, "float32"), tf.pow(_current_radius, 2))))
        learning_rate_matrix = tf.multiply(_current_learning_rate, neighbourhood_func)

        #Update all the weights
        multiplytiplier = tf.stack([tf.tile(tf.slice(
            learning_rate_matrix, np.array([i]), np.array([1])), [input_dim])
                                           for i in range(x*y)])
        delta = tf.multiply(
            multiplytiplier,
            tf.subtract(tf.stack([self._input for i in range(x*y)]), self._weights))                

        new_weights = tf.add(self._weights, delta)
        self._training = tf.assign(self._weights, new_weights)                                       

        #Initilize session and run it
        self._sess = tf.Session()
        initialization = tf.global_variables_initializer()
        self._sess.run(initialization)

  def train(self, input_vects):
      for iter_no in range(self._num_iter):
          for input_vect in input_vects:
              self._sess.run(self._training,
                            feed_dict={self._input: input_vect,
                                        self._iter_input: iter_no})

      self._centroid_matrix = [[] for i in range(self._x)]
      self._weights_list = list(self._sess.run(self._weights))
      self._locations = list(self._sess.run(self._locations))
      for i, loc in enumerate(self._locations):
          self._centroid_matrix[loc[0]].append(self._weights_list[i])

  def map_input(self, input_vectors):
      return_value = []
      for vect in input_vectors:
          min_index = min([i for i in range(len(self._weights_list))],
                          key=lambda x: np.linalg.norm(vect - self._weights_list[x]))
          return_value.append(self._locations[min_index])
      return return_value

  def _generate_index_matrix(self, x,y):
      return tf.constant(np.array(list(self._iterator(x, y))))

  def _iterator(self, x, y):
      for i in range(x):
          for j in range(y):
              yield np.array([i, j])

2.9.2


In [ ]:
som = SOM(x = 2, y = 1, input_dim=8, learning_rate=0.5, num_iter = 500, radius = 1.0)
som.train(normalized_performance);

In [ ]:
# print(som._weights_list)
print(som._locations)
cluster_centers_df = pd.DataFrame(data=som._weights_list,columns=data_performance.iloc[:,:-1].columns)
cluster_centers_df.head()

In [ ]:
cluster_assignment = som.map_input(normalized_performance)
data_performance["cluster_som"] = cluster_assignment
data_performance["cluster"] = data_performance.apply(lambda x: "good" if x["cluster_som"][0]==0 else "weak",axis=1)
data_performance = data_performance.drop(['cluster_som'],axis=1)
data_performance.head(5)

In [ ]:
data_performance_raw = pd.read_csv('/content/sample_data/Student_Performance.csv')
data_performance['student id'] = data_performance_raw['Student ID']

data_performance.head()

In [ ]:
data_performance.groupby("cluster").count()
# data_performance.describe()

## Visualize Clusters

In [ ]:
#pca
pca = PCA(n_components=2)
pca_performance = pd.DataFrame(pca.fit_transform(data_performance.iloc[:,:-3]))
pca_performance["cluster"] = data_performance["cluster"]
pca_performance.columns = ['pca0', 'pca1', 'cluster']
# setting the dimensions of the plot
fig, ax = plt.subplots(figsize=(15, 10))
# drawing the plot
sn.scatterplot(data=pca_performance, x='pca0', y='pca1', hue="cluster")
plt.show()

In [ ]:
data_engagement = pd.read_csv('/content/sample_data/Student_Engagement.csv')
data_engagement = data_engagement.drop(["Student ID"],axis=1)
data_engagement.head()

In [ ]:
msno.matrix(data_engagement,figsize=(10,3))


In [ ]:
continiousData = data_engagement.iloc[:,:-1]
continiousData.head()

fig, axes = plt.subplots(nrows=1,ncols=1)
fig.set_size_inches(50, 25)
sn.boxplot(data=continiousData,orient="v",ax=axes)

In [ ]:
# Correlation analysis
corrMatt = data_engagement.corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sn.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)

In [ ]:
# normalization
def minmax_scaler(data):
  scaler = MinMaxScaler()
  scaled = scaler.fit_transform(data)
  return scaled

normalized_engagement=minmax_scaler(data_engagement.iloc[:,:-1])
fig, axes = plt.subplots(nrows=1,ncols=1)
fig.set_size_inches(25, 15)
sn.boxplot(data=normalized_engagement,orient="v",ax=axes)

In [ ]:
som = SOM(x = 2, y = 1, input_dim=12, learning_rate=0.5, num_iter = 500, radius = 1.0)
som.train(normalized_engagement);

In [ ]:
# print(som._weights_list)
print(som._locations)
cluster_centers_df1 = pd.DataFrame(data=som._weights_list,columns=data_engagement.iloc[:,:-1].columns)
cluster_centers_df1.head()

In [ ]:
cluster_assignment1 = som.map_input(normalized_engagement)
data_engagement["cluster_som"] = cluster_assignment1
data_engagement["cluster"] = data_engagement.apply(lambda x: "high" if x["cluster_som"][0]==0 else "low",axis=1)
data_engagement = data_engagement.drop(['cluster_som'],axis=1)
data_engagement.head(10)

In [ ]:
data_engagement_raw = pd.read_csv('/content/sample_data/Student_Engagement.csv')
data_engagement['student id'] = data_engagement_raw['Student ID']

data_engagement.head(-1)

In [ ]:
data_engagement.groupby("cluster").count()
# data_performance.describe()

In [ ]:
#pca
pca = PCA(n_components=2)
pca_engagement = pd.DataFrame(pca.fit_transform(data_engagement.iloc[:,:-3]))
pca_engagement["cluster"] = data_engagement["cluster"]
pca_engagement.columns = ['pca0', 'pca1', 'cluster']
# setting the dimensions of the plot
fig, ax = plt.subplots(figsize=(15, 10))
# drawing the plot
sn.scatterplot(data=pca_engagement, x='pca0', y='pca1', hue="cluster")
plt.show()